## Importing the libraries.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Reading Datasets.

In [ ]:
path = 'C:/Users/Datasets'  # Replace with the actual folder path where the datasets are stored
dt_col_n = 'Date'
dfs = []
for fname in os.listdir(path):
    if fname.endswith(".xlsx"):
        f_path = os.path.join(path, fname)
        df = pd.read_excel(f_path)
        
        if dt_col_n in df.columns:
            dt_col = pd.to_datetime(df[dt_col_n])
            flt_data = df[(dt_col >= '2013-01-01') & (dt_col <= '2022-12-31')]
            dfs.append(flt_data)
        else:
            print(f"Date column '{dt_col_n}' not found in file: {fname}")

merge_df = pd.concat(dfs, ignore_index=True)

## Task 1: Wrangling, reshaping, EDA.

**Data Preprocessing**

In [ ]:
merge_df['Date'] = pd.to_datetime(merge_df['Date'])
df = merge_df.sort_values('Date')

In [ ]:
df.info()

Missing values of 'Best of' can be filled by checking the number of set played in that match. So here, set 3 and 4 if they are null, means the number of set played is either 2 or 3. Which means the match was based on best of 3. Again, if set 4 and 5 are not null, means number of set played is either 4 or 5. Which means the match was based on best of 5.

In [ ]:
df.loc[df[['W3', 'W4']].isna().all(axis=1), 'Best of'] = 3
df.loc[df[['W4']].isna().all(axis=1), 'Best of'] = 3

df.loc[df[['W4', 'W5']].notnull().all(axis=1), 'Best of'] = 5
df.loc[df[['W5']].notnull().all(axis=1), 'Best of'] = 5

In [ ]:
def fill_mean(rfr, col):
    means = df.groupby(rfr)[col].transform('mean')
    df[col].fillna(means, inplace=True)

In [ ]:
fill_mean('Winner','WRank')
fill_mean('Loser','LRank')
fill_mean('Winner','WPts')
fill_mean('Loser','LPts')

In [ ]:
df = df.dropna(subset=['WRank', 'LRank'], axis=0)

**Data Cleaning**

In [ ]:
print('Duplicate:',df.duplicated().sum())
df = df.drop_duplicates()

In [ ]:
df.describe()

Plotting Outliers.

In [ ]:
plt.subplot(2,2,1)
plt.subplots_adjust(left=.125,bottom=-1,right=.9,top=1,wspace=.2,hspace=.2)
plt.boxplot(df['WRank'])
plt.xlabel('WRank')

plt.subplot(2,2,2)
plt.subplots_adjust(left=.005,bottom=-1,right=.9,top=1,wspace=.2,hspace=.2)
plt.boxplot(df['WPts'])
plt.xlabel('WPts')

plt.subplot(2,2,3)
plt.subplots_adjust(left=.125,bottom=-1,right=.9,top=1,wspace=.2,hspace=.2)
plt.boxplot(df['LRank'])
plt.xlabel('LRank')

plt.subplot(2,2,4)
plt.subplots_adjust(left=.005,bottom=-1,right=.9,top=1,wspace=.2,hspace=.2)
plt.boxplot(df['LPts'])
plt.xlabel('LPts')

plt.show()

Defining function to identify and remove outliers. IQR method is used here to detect outliers. Calculate the difference between the 75th percentile (q3) and the 25th percentile (q1) of the data. **iqr = q3 - q1** Then multiply the IQR by a 1.5 to determine the lower and upper boundaries. **l_b = q1 - 1.5xiqr** and **u_b = q3 + 1.5xiqr** Extracting the indexes of outliers by checking the data and finding data less that lower boundary and higher than upper boundary. Finally removing the outliers by detecting the indexes.

In [ ]:
def outliers(dfr, col):
    q1 = dfr[col].quantile(.25)
    q3 = dfr[col].quantile(.75)
    iqr = q3 - q1
    
    l_b = q1 - 1.5*iqr
    u_b = q3 + 1.5*iqr
    
    ls = dfr.index[(dfr[col]<l_b) | (dfr[col]>u_b)]
    
    return ls

def remove(dfr, lst):
    ls = sorted(set(lst))
    dfs = dfr.drop(ls)
    return dfs

Checking the columns having outliers and storing by extracting the index calling 'outliers' function.

In [ ]:
i_ls = []
for f in ['WRank','LRank','WPts','LPts']:   
    i_ls.extend(outliers(df, f))

Removing the outliers and creating new dataframe with clean data.

In [ ]:
dfc = remove(df, i_ls)

In [ ]:
dfc.describe()

In [ ]:
dfc['OverallSequence'] = dfc.groupby(dfc['Date'].dt.year)['ATP'].rank(method='min')

**Data Visualization**

In [ ]:
plt.hist(dfc['ATP'])
plt.xlabel('ATP')
plt.ylabel('Frequency')
plt.title('Distribution of ATP')
plt.show()

In [ ]:
s_s_counts = df.groupby(['Series', 'Surface']).size().unstack()
s_s_counts.plot(kind='bar')
plt.xlabel('Series')
plt.ylabel('Count')
plt.title('Count of Tournaments by Series and Surface')
plt.xticks(rotation=45)
plt.legend(title='Surface')
plt.show()

In [ ]:
plt.pie(dfc['Surface'].value_counts(), labels=dfc['Surface'].unique(), autopct='%1.1f%%')
plt.title('Distribution of Surface types')
plt.show()

In [ ]:
series_counts = df['Series'].value_counts()
plt.pie(series_counts, labels=series_counts.index, autopct='%1.1f%%', wedgeprops=dict(width=0.3))
plt.title('Proportion of Tournaments by Series')
plt.show()

In [ ]:
f_data = dfc[dfc['Best of'] == 3]
s_counts = f_data['Series'].value_counts()

plt.bar(s_counts.index, s_counts.values)
plt.xlabel('Series')
plt.ylabel('Count')
plt.title('Series Played (Best of 3)')
plt.xticks(rotation=90)
plt.show()

In [ ]:
avg_odds = merge_df.groupby('Surface')['B365W'].mean()
plt.pie(avg_odds, labels=avg_odds.index, autopct='%1.1f%%')
plt.title('Average Betting Odds by Surface')
plt.show()

In [ ]:
plt.pie(dfc['Court'].value_counts(), labels=dfc['Court'].unique(), autopct='%1.1f%%')
plt.title('Distribution of Court Types')
plt.show()

In [ ]:
r_c_counts = dfc.groupby(['Round', 'Court']).size().reset_index(name='count')
sns.barplot(x='Round', y='count', hue='Court', data=r_c_counts)
plt.xlabel('Rounds')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.title('Rounds Played in Different Courts')
plt.show()

In [ ]:
sns.barplot(data=df, x='Round', y='WPts')
plt.xlabel('Round')
plt.ylabel('Points')
plt.title('Distribution of Points by Round')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.bar(dfc['Round'], dfc['WPts'], label='WPts', color='black')
plt.bar(dfc['Round'], dfc['LPts'], bottom=dfc['WPts'], label='LPts',color='yellow')
plt.xticks(rotation=45)
plt.xlabel('Round')
plt.ylabel('Points')
plt.title('Points by Round')
plt.legend()
plt.show()

 ## Task 2: Analysis questions and plotting

#### Who are the top 10 players by total wins in the dataset, and how many wins do they have? Plot this

Firstly, the total wins of the winner is calculated by counting the number of times their name appears in winner list and the count is stored in 'Total' column. Then the top 10 players with the highest number of wins is extracted. The players name and their wins is visualized using bar chart.

In [ ]:
win = dfc['Winner'].value_counts().reset_index(name='Total')
win = win.rename(columns={'index': 'Winner'})

top_10_w = win.head(10)
plt.figure(figsize=(7, 5))
plt.bar(top_10_w['Winner'], top_10_w['Total'], width=.5)
for i, v in enumerate(top_10_w['Total']):
    plt.text(i, v, str(v), ha='center', va='bottom')
plt.xlabel('Player')
plt.ylabel('Total Wins')
plt.title('Top 10 Players by Total Wins')
plt.xticks(rotation=45)
plt.show()

#### Which players have won the most Grand Slam matches and titles in the last 10 years? Plot this

Here, we'll extract all the winners who won the most 'Grand Slam' title. The titles won by the players are stored in the 'Series' column. So while extracting, we check the rows containing 'Grand Slam' and extract the players name. In the meantime, we also calculate the number of titles won by a particular player using value_count(). All the comparatable data according to condition is stored in the form of dataframe. The dataframe is then ploted using bar chart for visualization.

In [ ]:
grand_slam = dfc.loc[dfc['Series']=='Grand Slam',['Winner']].value_counts()
grand_slam.head(10).plot(kind='bar')

#### Who are the top 10 players according to the largest number of First Round tournament losses across all 10 years? Plot this.

Firstly, the total loss of the loser is calculated by counting the number of times their name appears in loser list and the count is stored in 'Total' column. Then the top 10 players with the highest number of loss is extracted. The players name and their loses is visualized using bar chart.

In [ ]:
los = dfc['Loser'].value_counts().reset_index(name='Total')
los = los.rename(columns={'index': 'Loser'})

top_10_l=los.head(10)
plt.figure(figsize=(7, 5))
plt.bar(top_10_l['Loser'], top_10_l['Total'], width=.5)
for i, v in enumerate(top_10_l['Total']):
    plt.text(i, v, str(v), ha='center', va='bottom')
plt.xlabel('Player')
plt.ylabel('Total Losses')
plt.title('Top 10 Players by Total Losses')
plt.xticks(rotation=45)
plt.show()

#### Identify the 5 biggest upsets for each year in the dataset based on ranking differentials. List player names, rankings, winner/loser, score, and tournament name and what the difference in the rankings was at the time.

To get the ranking difference, the rank of the winner and the loser is substracted and stored in 'DiffRank' column. The year from the date is extracted and stored in 'Year' column. The data are sorted with year in ascending and ranking difference in descending order. The data is again grouped with respect to year. Based on ranking difference, the top 5 upsets with the required informations are displayed.

In [ ]:
up_df = dfc
up_df['DiffRank'] = up_df['WRank'] - up_df['LRank']
up_df['Year'] = up_df['Date'].dt.year
sort_df = up_df.sort_values(['Year', 'DiffRank'], ascending=[True, False])
up_5 = sort_df.groupby('Year')
up_list = up_5[['Year', 'DiffRank', 'Winner', 'WRank', 'WPts', 'Loser', 'LRank', 'LPts', 'Tournament']]
up_list.head().reset_index()

#### Who were the top 10 players at year-end in 2017? How have their rankings changed over the period of 2013 to 2022? Plot this.

Firstly, all the data of the year 2017 is being extracted and stored in 'pl17_df'. Then the winner are grouped with respect to their winning ranks. Then the ranks are sorted in ascending order to get the highest rank on top. From there, the top 10 players are extracted. After which, we need to present their ranking throughout the period of 2013-2022. So we check their names in the winner list of all the data 2013-2022 and plot bar graph individually for all the players for better visualization of their performance

In [ ]:
pl17_df = dfc[dfc['Date'].dt.year == 2017]
rank_df = pl17_df.groupby('Winner')['WRank'].agg('max').reset_index()
rank_df_sorted = rank_df.sort_values('WRank', ascending=True)
top_plr = rank_df_sorted.head(10)
top_plr_2017 = dfc[dfc['Winner'].isin(top_plr['Winner'])]

for player in top_plr_2017['Winner'].unique():
    plr_data = top_plr_2017[top_plr_2017['Winner'] == player]
    plt.figure(figsize=(7, 5))
    plt.bar(plr_data['Year'], plr_data['WRank'], label=player, width=.5)
    plt.xlabel('Year')
    plt.ylabel('Ranking')
    plt.title('Rankings of '+player+' (2013-2022)')
    plt.legend()
    plt.show()

## Task 3: Advanced analysis questions

#### Which tournaments have had on average the most upsets (where a lower-ranked player defeated a higherranked player)? List the top 10 and plot their averages.

Firstly, we calculate the ranking difference where the loser defeated the winner. Then we group the tournament and calculate the mean of the upset ranking difference. Then we sort the values in descending order to get thw highest upsets on top. then we extract the top 10 tournament with highest upsets. Bar graph is used to visualize the data.

In [ ]:
ups_df = dfc
ups_df['DiffRank'] = ups_df['LRank'] - ups_df['WRank']
ups_avg = ups_df.groupby('Tournament')['DiffRank'].mean().reset_index()
ups_avg_sorted = ups_avg.sort_values('DiffRank', ascending=False)
top10_tournaments = ups_avg_sorted.head(10)

plt.figure(figsize=(7, 5))
plt.bar(top10_tournaments['Tournament'], top10_tournaments['DiffRank'], width=.5)
plt.xlabel('Tournament')
plt.ylabel('Average Upset Rate')
plt.title('Tournaments with the Highest Average Upset Rates')
plt.xticks(rotation=90)
plt.show()

#### Determine who the top 10 ranked players were at the end of 2022. Then calculate their head-to-head win-loss record against each other for all the matches they played in 2022. Present this result.

Sorting the ranks of both the winner and loser in the year 2022 and store the top 10 unique players data. And the data of 2022 is also stored separately. Then the data of the top 10 winner and loser is checked with all the data of 2022. then their  head-to-head win-loss record against each other is presented.

In [ ]:
top10_win = dfc[dfc['Date'].dt.year == 2022].sort_values('WRank').head(10)['Winner'].unique().tolist()
top10_los = dfc[dfc['Date'].dt.year == 2022].sort_values('LRank').head(10)['Loser'].unique().tolist()
data_2022 = dfc[dfc['Date'].dt.year == 2022]
mtc_top10 = data_2022[data_2022['Winner'].isin(top10_win) & data_2022['Loser'].isin(top10_los)]
vs = mtc_top10.groupby(['Winner', 'Loser']).size().unstack().fillna(0)
print(vs)

#### List the top 5 players who had the longest winning streaks between 2013 – 2022. List their names, the lengths of their winning streaks and the year(s) in which they occurred.

Firstly, streak of each player in a particular is calculated, the number of times they won in each year. Then we sort the players according to their streak in descending order to get the highest on top and store the top 5 highest player's streak. The data is then presented.

In [ ]:
temp_df = dfc
temp_df['Year'] = temp_df['Date'].dt.year
win_strk = temp_df.groupby(['Winner', 'Year']).size().reset_index(name='Streak')
top_5_streaks = win_strk.sort_values('Streak', ascending=False).head(5).reset_index()
print(top_5_streaks[['Winner', 'Streak', 'Year']])

#### In tennis, each set is played first to 6, but sometimes it is played to 7. A tiebreak is a set that someone wins 7-6 and is different to someone winning a set 7-5. Tiebreaks are stressful and some players perform better than others in tiebreaks. Count how many tiebreaks each player in the entire dataset has played. Then, calculate the percentage of tiebreaks that each player has won. List the top 10 players according to the percentage of tiebreaks won.

Firstly, we calculate the number of tiebreaker each player has played. A player has played tiebreaker if his/her point in a set reaches 7. So we check all the winner's all the set they played and store it. Same for the loser. Then we concate the data, group it according to unique players and add the number of tiebreaker they played. To calculate the number of tie breaker won by a player, we need to check their loser's point. Because a tiebreaker is won at a point(7-6). There we get the player the won a tiebreaker. To calculate the winning percentage, we take the number of tiebreaker played by a player and the number of win and evaluate. We then sor the data in descending order and extract the top 10 players record.

In [ ]:
df = dfc[['Year','Winner','Loser','W1','L1','W2','L2','W3','L3','W4','L4','W5','L5']]

win_pld = df.groupby('Winner')[['W1','W2','W3','W4','W5']].apply(lambda x: (x == 7).sum()).reset_index()
win_pld['Tiebreaks Played'] = win_pld['W1']+win_pld['W2']+win_pld['W3']+win_pld['W4']+win_pld['W5']
win_pld = win_pld.drop(columns=['W1','W2','W3','W4','W5'])
win_pld.rename(columns = {'Winner':'Player'}, inplace = True)

los_pld = dfc.groupby('Loser')[['L1','L2','L3','L4','L5']].apply(lambda x: (x == 7).sum()).reset_index()
los_pld['Tiebreaks Played'] = los_pld['L1']+los_pld['L2']+los_pld['L3']+los_pld['L4']+los_pld['L5']
los_pld = los_pld.drop(columns=['L1','L2','L3','L4','L5'])
los_pld.rename(columns = {'Loser':'Player'}, inplace = True)

tb_pld = pd.concat([win_pld, los_pld])
tb_pld = tb_pld.groupby('Player')['Tiebreaks Played'].sum().reset_index()


m1 = df[(df['W1'] == 7) & (df['L1'] == 6)].groupby('Winner').size().reset_index(name='m1')
m2 = df[(df['W2'] == 7) & (df['L2'] == 6)].groupby('Winner').size().reset_index(name='m2')
m3 = df[(df['W3'] == 7) & (df['L3'] == 6)].groupby('Winner').size().reset_index(name='m3')
m4 = df[(df['W4'] == 7) & (df['L4'] == 6)].groupby('Winner').size().reset_index(name='m4')
m5 = df[(df['W5'] == 7) & (df['L5'] == 6)].groupby('Winner').size().reset_index(name='m5')
tb_won = pd.concat([m1, m2, m3, m4, m5])
tb_won['Tiebreaks Won'] = tb_won['m1']+tb_won['m2']+tb_won['m3']+tb_won['m4']+tb_won['m5']
tb_won = tb_won.drop(columns=['m1','m2','m3','m4','m5'])
tb_won = tb_won.groupby('Winner')['Tiebreaks Won'].size().reset_index()
tb_won.rename(columns = {'Winner':'Player'}, inplace = True)

tb_data = pd.merge(tb_pld, tb_won, on='Player', how='left')
tb_data.fillna(0, inplace=True)
tb_data['Tiebreaks Won'] = tb_data['Tiebreaks Won'].astype('int64')
tb_data['Percentage Won'] = round(((tb_data['Tiebreaks Won'] / tb_data['Tiebreaks Played']) * 100),2)
tb_data.fillna(0, inplace=True)

top10_plrs = tb_data.sort_values('Percentage Won', ascending=False).head(10)
print(top10_plrs)

## Task 4: Open questions and analyses

#### Come up with 3 more questions of your own. Try to demonstrate the usage of more advanced data wrangling functionalities as you answer the questions like group by, pivots etc… Create several plots.


##### Display the top player in each year who won the most number of matches.

Firstly, calculate the number of matches a winner won in each year and store in a dataframe with a new column 'Match Won'. Then we group the data with respect to year and select the players data with highest 'Match Won'. Then we create a dataframe that stores only data of player with maximum Match Won each year, which is extracted by comparing data containing max Match Winner players and the data with winners total win. The dataframe with resultant data is than visualized using bar graph.

In [ ]:
ttl = dfc.groupby(['Year', 'Winner']).size().reset_index(name='Match Won')

max_ttl = ttl.groupby('Year')['Match Won'].idxmax()
top_plrs = ttl.loc[max_ttl]

plt.figure(figsize=(7, 5))
plt.bar(top_plrs['Winner'], top_plrs['Match Won'], width=.5)
plt.xlabel('Player')
plt.ylabel('Number of Match')
plt.title('Top Player by Match Won per Year')
plt.xticks(rotation=90)
plt.show()

##### Player that has the highest winning percentage in matches played on grass?

Here, we extract all the data of matches played where surface was grass. then we count the number of times a player won with value_counts(normalize=True)x100 which returns frequency of win, corresponding to win percentage. Then we sort the data in descending order to get the highest on top. Then we extract the top 10 and visualize their performance using horizontal bar graph.

In [ ]:
grass_matches = dfc[dfc['Surface'] == 'Grass']
win_per = (grass_matches['Winner'].value_counts(normalize=True)*100).round(2).reset_index()
win_per.columns = ['Player', 'Winning Percentage']
win_per = win_per.sort_values('Winning Percentage', ascending=False)
plt.figure(figsize=(10, 6))
plt.barh(win_per['Player'][:10], win_per['Winning Percentage'][:10])
plt.xlabel('Winning Percentage')
plt.ylabel('Player')
plt.title('Top 10 Players with the Highest Winning Percentages on Grass')
plt.show()

##### How many matches that has been played each year

Since our data contains detail of matches with date month year, it means per data represents matches played over the period 2013-2022. So we group all the data according to year and calculate size to get the number of matches played each year and stored with new column 'Matches'. Then tha data is visualized using plot chart.

In [ ]:
matches_by_year = dfc.groupby('Year').size().reset_index(name='Matches')
plt.figure(figsize=(7, 5))
plt.plot(matches_by_year['Year'], matches_by_year['Matches'], marker='o')
plt.xlabel('Year')
plt.ylabel('Number of Matches')
plt.title('Number of Matches Played Each Year')
plt.show()